<a href="https://colab.research.google.com/github/datasigntist/tensorflowCodes/blob/master/tensorflow101_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
import functools 
import numpy as np 
import tensorflow as tf 
import pandas as pd

LABEL_COLUMN = "Survived"

TRAIN_DATA_URL=  "https://raw.githubusercontent.com/datasigntist/datasetsForTraining/master/train.csv" 

train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL) 

def get_dataset(file_path, **kwargs):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=5, # Artificially small to make examples easier to show.
      label_name=LABEL_COLUMN,
      na_value="?",
      num_epochs=1,
      ignore_errors=True, 
      **kwargs)
  return dataset

def show_batch(dataset):
  for batch, label in dataset.take(1):
    for key, value in batch.items():
      print("{:20s}: {}".format(key,value.numpy()))

titanicDataSet = get_dataset(train_file_path)

class extractFeaturesFromName(object):
  def __call__(self, features, labels):
    namesOfPassengers = features['Name']
    lastName_feature = tf.map_fn(lambda feat: tf.strings.strip(tf.strings.split(feat,",")[0]) ,namesOfPassengers)
    firstName_feature = tf.map_fn(lambda feat: tf.strings.strip(tf.strings.split(tf.strings.strip(tf.strings.split(feat,",")[1]),".")[1]) ,namesOfPassengers)
    title_feature  = tf.map_fn(lambda feat: tf.strings.strip(tf.strings.split(tf.strings.strip(tf.strings.split(feat,",")[1]),".")[0]) ,namesOfPassengers)
    features['last_name'] = lastName_feature
    features['first_name'] = firstName_feature
    features['title_feature'] = title_feature
    return features, labels

class replaceMissingValuesInAge(object):
  def __init__(self, meanAge):
    self.meanAge = meanAge

  def __call__(self, features, labels):
    ageOfPassengers = features['Age']
    ageOfPassengers = tf.map_fn(lambda age: (age if age>0 else self.meanAge) ,ageOfPassengers)
    features['Age'] = ageOfPassengers
    return features, labels

class PackNumericFeatures(object):
  def __init__(self, names):
    self.names = names

  def __call__(self, features, labels):
    numeric_features = [features.pop(name) for name in self.names]
    numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
    numeric_features = tf.stack(numeric_features, axis=-1)
    features['numeric'] = numeric_features

    return features, labels

desc = pd.read_csv(train_file_path)["Age"].describe()
MEAN = np.float32(np.array(desc.T['mean']))

NUMERIC_FEATURES = ['Age','SibSp','Parch']

CATEGORIES = {
    'Sex': ['male', 'female']
}

newtitanicDataSet = titanicDataSet.map(extractFeaturesFromName())
newtitanicDataSet = newtitanicDataSet.map(replaceMissingValuesInAge(MEAN))
newtitanicDataSet = newtitanicDataSet.map(PackNumericFeatures(NUMERIC_FEATURES))

numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=None, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]

categorical_columns = []
for feature, vocab in CATEGORIES.items():
  cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
  categorical_columns.append(tf.feature_column.indicator_column(cat_col))

preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)

model = tf.keras.Sequential([
  preprocessing_layer,
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(1),
])

model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])

train_data = newtitanicDataSet.shuffle(500)
model.fit(train_data, epochs=20)

test_loss, test_accuracy = model.evaluate(train_data)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

predictions = model.predict(train_data)

# Show some results
for prediction, survived in zip(predictions[:10], list(train_data)[0][1][:10]):
  prediction = tf.sigmoid(prediction).numpy()
  print("Predicted survival: {:.2%}".format(prediction[0]),
        " | Actual outcome: ",
        ("SURVIVED" if bool(survived) else "DIED"))

Epoch 1/20
179/179 [==============================] - 0s 2ms/step - loss: 0.6454 - accuracy: 0.6409
Epoch 2/20
179/179 [==============================] - 0s 2ms/step - loss: 0.5561 - accuracy: 0.7464
Epoch 3/20
179/179 [==============================] - 0s 2ms/step - loss: 0.5494 - accuracy: 0.7666
Epoch 4/20
179/179 [==============================] - 0s 2ms/step - loss: 0.5099 - accuracy: 0.7935
Epoch 5/20
179/179 [==============================] - 0s 2ms/step - loss: 0.4844 - accuracy: 0.7980
Epoch 6/20
179/179 [==============================] - 0s 2ms/step - loss: 0.5159 - accuracy: 0.7890
Epoch 7/20
179/179 [==============================] - 0s 2ms/step - loss: 0.4955 - accuracy: 0.8002
Epoch 8/20
179/179 [==============================] - 0s 2ms/step - loss: 0.4877 - accuracy: 0.8013
Epoch 9/20
179/179 [==============================] - 0s 2ms/step - loss: 0.4841 - accuracy: 0.8013
Epoch 10/20
179/179 [==============================] - 0s 2ms/step - loss: 0.4968 - accuracy: 0.7946